In [3]:
import glob
from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("../config/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

In [10]:
# GCP 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [7]:
credentials.project_id

'data-analysis-413914'

In [18]:
# 데이터 조회 쿼리
sql = f"""
SELECT *
FROM kobis.daily_box_office
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
df = query_job.to_dataframe()
df.head()

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/data-analysis-413914/jobs?prettyPrint=false: Access Denied: Project data-analysis-413914: User does not have bigquery.jobs.create permission in project data-analysis-413914.

Location: None
Job ID: 31637717-976f-4622-8db7-2f8f32a98de7
